In [4]:
!pip3 install fastapi uvicorn gradio requests nest_asyncio




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 10.3 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [5]:
import nest_asyncio
nest_asyncio.apply()


In [6]:
def openai_model(prompt, history):
    return "This is a dummy response from OpenAI."

def gemini_model(prompt, history):
    return "This is a dummy response from Gemini."

def claude_model(prompt, history):
    return "This is a dummy response from Claude."

model_map = {
    "openai": openai_model,
    "gemini": gemini_model,
    "claude": claude_model
}

def get_llm(model_name):
    return model_map.get(model_name, model_map["openai"])


In [7]:
class ChatAgent:
    def __init__(self):
        self.history = []

    def get_response(self, message, llm):
        self.history.append({"user": message})
        reply = llm(message, self.history)
        self.history.append({"bot": reply})
        return reply


In [8]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from threading import Thread

app = FastAPI()
agent = ChatAgent()

class ChatRequest(BaseModel):
    message: str
    model: str = "openai"

@app.post("/chat")
async def chat(req: ChatRequest):
    llm = get_llm(req.model)
    reply = agent.get_response(req.message, llm)
    return {"response": reply}

def run_api():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Start FastAPI in background thread
Thread(target=run_api).start()


INFO:     Started server process [47661]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:49421 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:49421 - "GET /favicon.ico HTTP/1.1" 404 Not Found


In [10]:
import gradio as gr
import requests

def chat_fn(user_message, history, model):
    try:
        resp = requests.post("http://127.0.0.1:8000/chat", json={
            "message": user_message,
            "model": model
        })
        reply = resp.json()["response"]
    except Exception as e:
        reply = "Server error: " + str(e)
    history.append((user_message, reply))
    return history, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type your message")
    model = gr.Dropdown(["openai", "gemini", "claude"], value="openai", label="Choose Model")
    btn = gr.Button("Send")

    state = gr.State([])

    btn.click(chat_fn, [msg, state, model], [chatbot, state])

demo.launch()



/var/folders/gd/m3mhxdp57lj4yt616w0pdgvh0000gn/T/ipykernel_47661/2827212334.py:17: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
